----

## Öncelikle bakalım bu residual bloklar hangi model tiplerinde kullanılabilirmiş.Aşağıya 3 adet model bırakıyorum.Örnek olarak inceleyeceğiz.

----

In [1]:
import torch
import torch.nn as nn
def make_norm(norm: str, ch: int):
    norm = norm.lower()
    if norm == "bn":
        return nn.BatchNorm2d(ch)
    if norm == "gn":
        g = min(32, ch)
        while ch % g != 0 and g > 2:
            g //= 2
        if ch % g != 0:
            g = 2 if (ch % 2 == 0) else 1
        return nn.GroupNorm(g, ch)
    if norm == "none":
        return nn.Identity()
    raise ValueError("norm 'bn', 'gn', 'none' olmalı.")

def make_act(act: str):
    act = act.lower()
    if act == "relu":
        return nn.ReLU(inplace=True)
    if act == "silu":
        return nn.SiLU(inplace=True)
    raise ValueError("act 'relu' veya 'silu' olmalı.");

# Klasik ResNet-stil backbone (ImageNet/CIFAR tarzı)

In [ ]:
class BottleneckResidualV1(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, stride: int = 1,
                 expansion: int = 4, norm: str = "bn", act: str = "relu"):
        super().__init__()
        if out_ch % expansion != 0:
            raise ValueError("out_ch, expansion'a bölünebilir olmalı.")
        mid = out_ch // expansion

        self.act = make_act(act)

        # Main path F(x)
        self.conv1 = nn.Conv2d(in_ch, mid, 1, bias=False)
        self.bn1   = make_norm(norm, mid)

        self.conv2 = nn.Conv2d(mid, mid, 3, stride=stride, padding=1, bias=False)
        self.bn2   = make_norm(norm, mid)

        self.conv3 = nn.Conv2d(mid, out_ch, 1, bias=False)
        self.bn3   = make_norm(norm, out_ch)

        # Skip path
        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False),
                make_norm(norm, out_ch),
            )

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)

        y = self.act(self.bn1(self.conv1(x)))
        y = self.act(self.bn2(self.conv2(y)))
        y = self.bn3(self.conv3(y))

        out = self.act(skip + y)
        return out
    
class Model_ResNetLike(nn.Module):
    """
    Örnek-1 model: Klasik "stem + 3 stage" bottleneck residual backbone
    """
    def __init__(self, num_classes=10, norm="bn", act="relu"):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=1, padding=1, bias=False),
            make_norm(norm, 64),
            make_act(act),
        )

        # Stage1: 64 -> 256
        self.s1 = nn.Sequential(
            BottleneckResidualV1(64, 256, stride=1, norm=norm, act=act),
            BottleneckResidualV1(256, 256, stride=1, norm=norm, act=act),
        )

        # Stage2: 256 -> 512 (downsample)
        self.s2 = nn.Sequential(
            BottleneckResidualV1(256, 512, stride=2, norm=norm, act=act),
            BottleneckResidualV1(512, 512, stride=1, norm=norm, act=act),
        )

        # Stage3: 512 -> 1024 (downsample)
        self.s3 = nn.Sequential(
            BottleneckResidualV1(512, 1024, stride=2, norm=norm, act=act),
            BottleneckResidualV1(1024, 1024, stride=1, norm=norm, act=act),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.s1(x)
        x = self.s2(x)
        x = self.s3(x)
        return self.head(x)

# Attention  entegre edilmiş bottleneck residual

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleSE(nn.Module):
    """
    Basit SE: örnek attention yerine (CBAM/Coord yerine burada minimal)
    """
    def __init__(self, ch: int, r: int = 16):
        super().__init__()
        hid = max(4, ch // r)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(ch, hid, 1)
        self.fc2 = nn.Conv2d(hid, ch, 1)

    def forward(self, x):
        s = self.pool(x)
        s = F.relu(self.fc1(s), inplace=True)
        s = torch.sigmoid(self.fc2(s))
        return x * s


class BottleneckResidual_Attn(nn.Module):
    """
    Bottleneck + attention (F(x) içine)
    """
    def __init__(self, in_ch, out_ch, stride=1, expansion=4,
                 norm="bn", act="relu", attention: nn.Module = None):
        super().__init__()
        if out_ch % expansion != 0:
            raise ValueError("out_ch, expansion'a bölünebilir olmalı.")
        mid = out_ch // expansion
        self.act = make_act(act)

        self.conv1 = nn.Conv2d(in_ch, mid, 1, bias=False)
        self.bn1   = make_norm(norm, mid)

        self.conv2 = nn.Conv2d(mid, mid, 3, stride=stride, padding=1, bias=False)
        self.bn2   = make_norm(norm, mid)

        self.conv3 = nn.Conv2d(mid, out_ch, 1, bias=False)
        self.bn3   = make_norm(norm, out_ch)

        self.attn = attention if attention is not None else nn.Identity()

        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False),
                make_norm(norm, out_ch),
            )

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)

        y = self.act(self.bn1(self.conv1(x)))
        y = self.act(self.bn2(self.conv2(y)))
        y = self.bn3(self.conv3(y))
        y = self.attn(y)               # <-- attention burada

        out = self.act(skip + y)
        return out


class Model_AttnBackbone(nn.Module):
    """
    Örnek-2 model: Her stage'in sonunda attention'lı bottleneck residual
    """
    def __init__(self, num_classes=10, norm="bn", act="relu"):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            make_norm(norm, 32),
            make_act(act),
        )

        self.stage = nn.Sequential(
            BottleneckResidual_Attn(32, 128, stride=1, norm=norm, act=act, attention=SimpleSE(128)),
            BottleneckResidual_Attn(128, 256, stride=2, norm=norm, act=act, attention=SimpleSE(256)),
            BottleneckResidual_Attn(256, 512, stride=2, norm=norm, act=act, attention=SimpleSE(512)),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.stage(x)
        return self.head(x)


# 3) INVERTED BOTTLENECK (MBConv) + RESIDUAL (MobileNet/EfficientNet style)

In [ ]:
class MBConv(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1, expand_ratio=4,
                 norm="bn", act="silu", use_se=True):
        super().__init__()
        if stride not in (1, 2):
            raise ValueError("stride 1 veya 2 olmalı.")

        self.use_res = (stride == 1 and in_ch == out_ch)

        mid = in_ch * expand_ratio
        if expand_ratio == 1:
            mid = in_ch
            
        self.expand = nn.Identity()
        if expand_ratio != 1:
            self.expand = nn.Sequential(
                nn.Conv2d(in_ch, mid, 1, bias=False),
                make_norm(norm, mid),
                make_act(act),
            )

        # 2) depthwise
        self.dw = nn.Sequential(
            nn.Conv2d(mid, mid, 3, stride=stride, padding=1, groups=mid, bias=False),
            make_norm(norm, mid),
            make_act(act),
        )

        self.attn = SimpleSE(mid) if use_se else nn.Identity()
        self.project = nn.Sequential(
            nn.Conv2d(mid, out_ch, 1, bias=False),
            make_norm(norm, out_ch),
        )

    def forward(self, x):
        y = self.expand(x)
        y = self.dw(y)
        y = self.attn(y)     
        y = self.project(y)

        if self.use_res:
            return x + y
        return y


class Model_MobileLike(nn.Module):
    def __init__(self, num_classes=10, norm="bn"):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 24, 3, stride=2, padding=1, bias=False),
            make_norm(norm, 24),
            nn.SiLU(inplace=True),
        )

        self.blocks = nn.Sequential(
            MBConv(24, 24, stride=1, expand_ratio=2, norm=norm, act="silu", use_se=True),
            MBConv(24, 48, stride=2, expand_ratio=4, norm=norm, act="silu", use_se=True),
            MBConv(48, 48, stride=1, expand_ratio=4, norm=norm, act="silu", use_se=True),
            MBConv(48, 96, stride=2, expand_ratio=4, norm=norm, act="silu", use_se=True),
            MBConv(96, 96, stride=1, expand_ratio=4, norm=norm, act="silu", use_se=True),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(96, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        return self.head(x)

# Quick smoke test

In [12]:
if __name__ == "__main__":
    x = torch.randn(2, 3, 64, 64)

    m1 = Model_ResNetLike(num_classes=10)
    m2 = Model_AttnBackbone(num_classes=10)
    m3 = Model_MobileLike(num_classes=10)

    y1 = m1(x)
    y2 = m2(x)
    y3 = m3(x)

    print("ResNetLike:", y1.shape)
    print("AttnBackbone:", y2.shape)
    print("MobileLike:", y3.shape)

ResNetLike: torch.Size([2, 10])
AttnBackbone: torch.Size([2, 10])
MobileLike: torch.Size([2, 10])


----
----
----
----
----
----